In [1]:
import numpy as np
import pandas as pd 

#sklearn processing
from sklearn.model_selection import KFold

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv
/kaggle/input/feedback-prize-english-language-learning/train.csv
/kaggle/input/feedback-prize-english-language-learning/test.csv


In [2]:
sample_submission_raw = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv')
train_data_raw = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
test_data = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')

In [3]:
kf = KFold(n_splits=5)
kfolds_data = kf.split(train_data_raw)

partition = 1
for train_index, test_index in kfolds_data:
    train = pd.DataFrame(train_data_raw.values[train_index])
    train.columns = train_data_raw.columns

    test = pd.DataFrame(train_data_raw.values[test_index])
    test.columns = train_data_raw.columns
    
    train.to_csv(f'/kaggle/working/train_partition_{partition}.csv')
    test.to_csv(f'/kaggle/working/test_partition_{partition}.csv')
    
    partition +=1
    

In [4]:
train = pd.read_csv('/kaggle/working/train_partition_1.csv')
test = pd.read_csv('/kaggle/working/test_partition_1.csv')

In [5]:
def baseline_prediction(parameter):
    return train[parameter].median()
    
def get_prediction(parameter, model='baseline'):
    if model == 'baseline':
        return baseline_prediction(parameter)
    
def calc_rmse(parameter, model='baseline'):
    prediction = get_prediction(parameter, model)
    
    return np.sqrt(((train[parameter] - prediction)**2).mean())

parameters_to_predict = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']

rmse_scores = []
for parameter in parameters_to_predict:
    rmse_scores.append(calc_rmse(parameter))
    
np.mean(rmse_scores)

0.6653298094965044

In [6]:
test_data['cohesion'] = baseline_prediction('cohesion')
test_data['syntax'] = baseline_prediction('syntax')
test_data['vocabulary'] = baseline_prediction('vocabulary')
test_data['phraseology'] = baseline_prediction('phraseology')
test_data['grammar'] = baseline_prediction('grammar')
test_data['conventions'] = baseline_prediction('conventions')
test_data

,text_id,full_text,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,when a person has no experience on a job their...,3.0,3.0,3.0,3.0,3.0,3.0
1,000BAD50D026,Do you think students would benefit from being...,3.0,3.0,3.0,3.0,3.0,3.0
2,00367BB2546B,"Thomas Jefferson once states that ""it is wonde...",3.0,3.0,3.0,3.0,3.0,3.0


In [7]:
res = test_data.drop("full_text",axis=1)
res.to_csv("/kaggle/working/submission.csv",index=False)